In [43]:
import requests
from dotenv import load_dotenv
import os
import json
import pandas as pd
from datetime import datetime
from datetime import timezone
from dateutil import tz

# function to load secrets
def secretFunc():
	load_dotenv()
	global TRANSIT_URL
	global TRANSIT_API_KEY
	global STOPCODES
	global DIRECTIONS
	global OPERATORS
	global STOPNAMES
	global OPENWEATHER_API_KEY

	TRANSIT_URL = 'http://api.511.org/transit/StopMonitoring/'
	TRANSIT_API_KEY = os.environ['TRANSIT_API_KEY']
	OPENWEATHER_API_KEY = os.environ['OPENWEATHER_API_KEY']

	# update STOPCODES, OPERATORS, DIRECTIONS, STOPNAMES to add add'l stops

	STOPCODES = [13915, 13914, 14509, 14510]

	OPERATORS = ['SF','SF', 'SF', 'SF']

	DIRECTIONS = ['Inbound', 'Outbound', 'Inbound', 'Outbound']

	STOPNAMES = ['Stanyan', 'Stanyan', 'Folsom', 'Folsom']

	return()

# function to use 511org API to find next arrivals at each of the provided STOPCODES
def getNextTransit():
	secretFunc()
	arrivals = list()
	for i, stop in enumerate(STOPCODES):
		r = requests.get(TRANSIT_URL, 
						 params = {'agency': OPERATORS[i], 
								   'api_key': TRANSIT_API_KEY,
								   'stopcode': stop})
		
		content = json.loads(r.content)

		stopInfo = pd.DataFrame.from_records(content)
		stopInfo = stopInfo['ServiceDelivery']['StopMonitoringDelivery']['MonitoredStopVisit']

		for arrival in stopInfo:
			# Get arrivalTime from JSON content
			arrivalTime = arrival['MonitoredVehicleJourney']['MonitoredCall']['ExpectedArrivalTime']

			# Convert arrivalTime into 12h Pacific time
			utc_datetime = datetime.fromisoformat(arrivalTime[:-1])

			# Define the timezones
			utc_tz = tz.gettz('UTC')
			pacific_tz = tz.gettz('America/Los_Angeles')
			currentTime = datetime.now(timezone.utc)

			# Set the UTC timezone for the datetime object
			utc_datetime = utc_datetime.replace(tzinfo = utc_tz)

			# Convert the datetime object to Pacific time
			pacific_datetime = utc_datetime.astimezone(pacific_tz)

			# Format the datetime object as a 12-hour time string
			time_str = pacific_datetime.strftime('%I:%M %p')
			
			# Calculte timeToArrival
			timeToArrival = pacific_datetime - currentTime
			timeToArrival = divmod(timeToArrival.seconds, 60)
			timeToArrival = f"{str(timeToArrival[0]).rjust(2,'0')}:{str(timeToArrival[1]).rjust(2,'0')}"

			# Get destination from JSON content
			destination = arrival['MonitoredVehicleJourney']['MonitoredCall']['DestinationDisplay']

			# Append [code, direction, time]  to list
			arrivals.append([STOPNAMES[i], DIRECTIONS[i], destination, time_str, timeToArrival, stop])

	# Convert nested list to DataFrame
	arrivals = pd.DataFrame(arrivals, columns=['stopnames', 'direction', 'destination','arrivalTime', 'timeToArrival', 'stopcode'])
	return(arrivals)


	''' 
	todo:
		- connect to RPLCD to print information
		- implement refreshment interval
	'''

# main function
if __name__ == '__main__':
	transitArrivals = getNextTransit()
	print(transitArrivals)

   stopnames direction           destination arrivalTime timeToArrival  \
0    Stanyan   Inbound     Caltrain/Ballpark    09:53 PM         01:54   
1    Stanyan   Inbound     Caltrain/Ballpark    10:06 PM         14:59   
2    Stanyan   Inbound     Caltrain/Ballpark    10:22 PM         30:11   
3    Stanyan   Inbound   Townsend and 5th St    11:55 PM        124:06   
4    Stanyan   Inbound   Townsend and 5th St    12:15 AM        144:10   
5    Stanyan   Inbound  Fourth St + Townsend    12:34 AM        162:13   
6    Stanyan  Outbound           Ocean Beach    10:02 PM         10:15   
7    Stanyan  Outbound           Ocean Beach    10:16 PM         24:23   
8    Stanyan  Outbound           Ocean Beach    10:30 PM         38:29   
9    Stanyan  Outbound           Ocean Beach    12:34 AM        162:46   
10    Folsom   Inbound     Caltrain/Ballpark    09:57 PM         05:48   
11    Folsom   Inbound   Embarcadero Station    09:59 PM         08:03   
12    Folsom   Inbound     Caltrain/Ba

In [45]:
test = transitArrivals.loc[0, 'arrivalTime':'timeToArrival']
test.loc['arrivalTime']

'09:53 PM'

In [46]:
inbound1 = transitArrivals.loc[0, 'arrivalTime':'timeToArrival']
inbound2 = transitArrivals.loc[1, 'arrivalTime':'timeToArrival']

print(inbound1.loc['arrivalTime'], inbound1.loc['timeToArrival'] + '\n' + inbound2.loc['arrivalTime'], inbound2.loc['timeToArrival'])

09:53 PM 01:54
10:06 PM 14:59


In [65]:
output1 = f" City: {transitArrivals.loc[0, 'arrivalTime']}"
output2 = f"Next: {transitArrivals.loc[1, 'arrivalTime']}"

print(output1, '\n', output2)
len(output1)

 City: 09:53 PM 
 Next: 10:06 PM


15

In [61]:
print(len('Humidity: 89%'))
print(len('Wind: 13.8 MPH'))

13
14


In [59]:
transitArrivals[transitArrivals['destination'].str.contains('Ocean Beach')]

,stopnames,direction,destination,arrivalTime,timeToArrival,stopcode
6,Stanyan,Outbound,Ocean Beach,10:02 PM,10:15,13914
7,Stanyan,Outbound,Ocean Beach,10:16 PM,24:23,13914
8,Stanyan,Outbound,Ocean Beach,10:30 PM,38:29,13914
9,Stanyan,Outbound,Ocean Beach,12:34 AM,162:46,13914
15,Folsom,Outbound,Ocean Beach,09:57 PM,05:13,14510
16,Folsom,Outbound,Ocean Beach,10:11 PM,19:19,14510
17,Folsom,Outbound,Ocean Beach,10:31 PM,39:19,14510
